In [46]:
%autoreload 2
from pathlib import Path

from generation import generate_agents_subpopulations, generate_agents_berlin_trips, berlinwise_col, \
    generate_nonberlin_trips, generate_trips_berlinwise
from reader import read_agents, read_trips, read_trips_berlinwise, read_trips_additional
from scenarios import BerlinScenario, SCENARIOS_PATH

In [3]:
def path(scenario: BerlinScenario, kind, file_name_suffix) -> Path:
    return SCENARIOS_PATH / scenario.value / kind / f"{scenario}.{file_name_suffix}"

Read in input csv (both from MATSim output and from Java analysis)

In [4]:
agents = read_agents(path(BerlinScenario.BASE, "output", "output_persons.csv.gz"))
agents

,executed_score,first_act_x,first_act_y,first_act_type,income,subpopulation,home-activity-zone
person,,,,,,,
10000001,90.723000,4.603071e+06,5.705496e+06,home_77400.0,2659.0,person,brandenburg
1000001,116.991525,4.655520e+06,5.752325e+06,home_91800.0,826.0,person,brandenburg
100002101,92.331500,4.628160e+06,5.828612e+06,home_36000.0,826.0,person,brandenburg
100002901,82.349915,4.628056e+06,5.830651e+06,home_48600.0,1847.0,person,brandenburg
100003501,130.202918,4.625913e+06,5.824592e+06,home_82200.0,1847.0,person,brandenburg
...,...,...,...,...,...,...,...
freight_997_87464-berlin,117.274442,4.603503e+06,5.808240e+06,freight,NaN,freight,NaN
freight_998_87464-berlin,116.469365,4.603503e+06,5.808240e+06,freight,NaN,freight,NaN
freight_999_87464-berlin,118.242244,4.603503e+06,5.808240e+06,freight,NaN,freight,NaN


In [5]:
trips_by_scenario = {scenario: read_trips(path(BerlinScenario.BASE, "output", "output_trips.csv.gz")) for
                     scenario in BerlinScenario}
trips_by_scenario[BerlinScenario.BASE]

,person,trip_number,dep_time,trav_time,wait_time,traveled_distance,euclidean_distance,main_mode,longest_distance_mode,modes,...,start_facility_id,start_link,start_x,start_y,end_facility_id,end_link,end_x,end_y,first_pt_boarding_stop,last_pt_egress_stop
trip_id,,,,,,,,,,,,,,,,,,,,,
10000001_1,10000001,1,09:17:00,00:17:42,00:00:00,8050,6787,car,car,walk-car-walk,...,NaN,144395,4.602586e+06,5.705900e+06,NaN,134075,4.597352e+06,5.710221e+06,NaN,NaN
10000001_2,10000001,2,09:39:42,00:27:43,00:00:00,18437,13779,car,car,walk-car-walk,...,NaN,134075,4.597352e+06,5.710221e+06,NaN,77361,4.583822e+06,5.712834e+06,NaN,NaN
10000001_3,10000001,3,10:19:25,00:48:35,00:00:00,34241,24989,car,car,walk-car-walk,...,NaN,77361,4.583822e+06,5.712834e+06,NaN,31841,4.608361e+06,5.708108e+06,NaN,NaN
10000001_4,10000001,4,12:02:00,00:18:43,00:00:00,10508,5899,car,car,walk-car-walk,...,NaN,31841,4.608361e+06,5.708108e+06,NaN,144395,4.603071e+06,5.705496e+06,NaN,NaN
10000001_5,10000001,5,13:05:43,00:42:46,00:00:00,17661,9403,car,car,walk-car-walk,...,NaN,144395,4.603071e+06,5.705496e+06,NaN,106140,4.609164e+06,5.698333e+06,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
freight_997_87464-berlin_1,freight_997_87464-berlin,1,06:01:16,00:26:07,00:00:00,13893,10437,freight,freight,walk-freight-walk,...,NaN,87167,4.603503e+06,5.808240e+06,NaN,54169,4.598759e+06,5.817537e+06,NaN,NaN
freight_998_87464-berlin_1,freight_998_87464-berlin,1,10:58:14,00:29:36,00:00:00,16478,10720,freight,freight,walk-freight-walk,...,NaN,87167,4.603503e+06,5.808240e+06,NaN,90464,4.593564e+06,5.812259e+06,NaN,NaN
freight_999_87464-berlin_1,freight_999_87464-berlin,1,02:54:09,00:21:21,00:00:00,10876,7147,freight,freight,walk-freight-walk,...,NaN,87167,4.603503e+06,5.808240e+06,NaN,88992,4.604471e+06,5.815321e+06,NaN,NaN


In [45]:
trips_berlinwise_by_scenario = {scenario: read_trips_berlinwise(path(scenario, "analysis", "trips_berlinwise.csv")) for
                                scenario in BerlinScenario}
trips_berlinwise_by_scenario[BerlinScenario.BASE]

,person,trip_number,modes,start_link,end_link,start_area,visited_areas,end_area
trip_id,,,,,,,,
10000001_1,10000001,1,walk-car-walk,144395,134075,brandenburg,brandenburg,brandenburg
10000001_2,10000001,2,walk-car-walk,134075,77361,brandenburg,brandenburg,brandenburg
10000001_3,10000001,3,walk-car-walk,77361,31841,brandenburg,brandenburg,brandenburg
10000001_4,10000001,4,walk-car-walk,31841,144395,brandenburg,brandenburg,brandenburg
10000001_5,10000001,5,walk-car-walk,144395,106140,brandenburg,brandenburg,brandenburg
...,...,...,...,...,...,...,...,...
freight_997_87464-berlin_1,freight_997_87464-berlin,1,walk-freight-walk,87167,54169,brandenburg,"brandenburg,berlin_outside_umweltzone,berlin_u...",berlin_umweltzone
freight_998_87464-berlin_1,freight_998_87464-berlin,1,walk-freight-walk,87167,90464,brandenburg,"brandenburg,berlin_outside_umweltzone",berlin_outside_umweltzone
freight_999_87464-berlin_1,freight_999_87464-berlin,1,walk-freight-walk,87167,88992,brandenburg,"brandenburg,berlin_outside_umweltzone",berlin_outside_umweltzone


In [ ]:
trips_additional_by_scenario = {scenario: read_trips_additional(path(scenario, "analysis", "trips_additional.csv")) for
                                scenario in BerlinScenario}
trips_additional_by_scenario[BerlinScenario.BASE]

In [48]:
read_trips_additional(path(BerlinScenario.BASE, "analysis", "trips_additional.csv"))

,person,trip_number,modes,last_link,last_area,completed,visited_link_count,car_berlin_a_main_street,car_berlin_a_side_street,car_berlin_a_other_street,car_berlin_b_main_street,car_berlin_b_side_street,car_berlin_b_other_street,car_brandenburg,pt_berlin_a,pt_berlin_b,pt_brandenburg
trip_id,,,,,,,,,,,,,,,,,
10000001_1,10000001,1,walk-car-walk,134075,brandenburg,True,25,0.0,0.0,0.0,0.0,0.0,0.0,7483.4,0.0,0.0,0.0
10000001_2,10000001,2,walk-car-walk,77361,brandenburg,True,57,0.0,0.0,0.0,0.0,0.0,0.0,17849.9,0.0,0.0,0.0
10000001_3,10000001,3,walk-car-walk,31841,brandenburg,True,107,0.0,0.0,0.0,0.0,0.0,0.0,33989.1,0.0,0.0,0.0
10000001_4,10000001,4,walk-car-walk,144395,brandenburg,True,28,0.0,0.0,0.0,0.0,0.0,0.0,10275.9,0.0,0.0,0.0
10000001_5,10000001,5,walk-car-walk,106140,brandenburg,True,63,0.0,0.0,0.0,0.0,0.0,0.0,16777.4,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pt_pt_X9---17528_700_9_3_Bus_veh_type_1,pt_pt_X9---17528_700_9_3_Bus_veh_type,1,car,pt_43894,berlin_outside_umweltzone,False,8,0.0,0.0,3839.3,0.0,0.0,2797.2,0.0,0.0,0.0,0.0
pt_pt_X9---17528_700_9_4_Bus_veh_type_1,pt_pt_X9---17528_700_9_4_Bus_veh_type,1,car,pt_43894,berlin_outside_umweltzone,False,8,0.0,0.0,3839.3,0.0,0.0,2797.2,0.0,0.0,0.0,0.0
pt_pt_X9---17528_700_9_5_Bus_veh_type_1,pt_pt_X9---17528_700_9_5_Bus_veh_type,1,car,pt_43894,berlin_outside_umweltzone,False,8,0.0,0.0,3839.3,0.0,0.0,2797.2,0.0,0.0,0.0,0.0


Generate temporary files that split the population into relevant, freight and ride

In [6]:
agent_subpopulations = generate_agents_subpopulations(agents, trips_by_scenario[BerlinScenario.BASE])
agent_subpopulations.relevant.to_csv(path(BerlinScenario.BASE, "analysis", "relevant_persons.csv"), sep=";")
agent_subpopulations.freight.to_csv(path(BerlinScenario.BASE, "analysis", "freight_persons.csv"), sep=";")
agent_subpopulations.ride.to_csv(path(BerlinScenario.BASE, "analysis", "ride_persons.csv"), sep=";")
agent_subpopulations.relevant

,executed_score,first_act_x,first_act_y,first_act_type,income,subpopulation,home-activity-zone,main_modes
person,,,,,,,,
10000001,90.723000,4.603071e+06,5.705496e+06,home_77400.0,2659.0,person,brandenburg,{car}
1000001,116.991525,4.655520e+06,5.752325e+06,home_91800.0,826.0,person,brandenburg,{bicycle}
100002101,92.331500,4.628160e+06,5.828612e+06,home_36000.0,826.0,person,brandenburg,{bicycle}
100002901,82.349915,4.628056e+06,5.830651e+06,home_48600.0,1847.0,person,brandenburg,{car}
100004901,110.151148,4.625238e+06,5.824173e+06,home_54600.0,2070.0,person,brandenburg,{car}
...,...,...,...,...,...,...,...,...
99994101,95.497982,4.632244e+06,5.826463e+06,home_60600.0,4329.0,person,brandenburg,{bicycle}
99995401,117.394194,4.631520e+06,5.826864e+06,home_83400.0,1142.0,person,brandenburg,{bicycle}
99995901,132.648340,4.627167e+06,5.827323e+06,home_77400.0,2332.0,person,brandenburg,"{walk, bicycle}"


Lists trips which are different in values of `modes`, `start_link` or `end_link`, even though it should be equal in our understanding

In [13]:
joined = trips_by_scenario[BerlinScenario.GR_HS][
    ["main_mode", "modes", "start_link", "end_link"]].join(
    trips_berlinwise_by_scenario[BerlinScenario.GR_HS][
        ["modes", "start_link", "end_link", "start_area", "visited_areas", "end_area"]],
    lsuffix="_output", rsuffix="_analysis",
    how="outer", validate="1:1")
joined

,main_mode,modes_output,start_link_output,end_link_output,modes_analysis,start_link_analysis,end_link_analysis,start_area,visited_areas,end_area
trip_id,,,,,,,,,,
10000001_1,car,walk-car-walk,144395,134075,walk-car-walk,144395,134075,brandenburg,brandenburg,brandenburg
10000001_2,car,walk-car-walk,134075,77361,walk-car-walk,134075,77361,brandenburg,brandenburg,brandenburg
10000001_3,car,walk-car-walk,77361,31841,walk-car-walk,77361,31841,brandenburg,brandenburg,brandenburg
10000001_4,car,walk-car-walk,31841,144395,walk-car-walk,31841,144395,brandenburg,brandenburg,brandenburg
10000001_5,car,walk-car-walk,144395,106140,walk-car-walk,144395,106140,brandenburg,brandenburg,brandenburg
...,...,...,...,...,...,...,...,...,...,...
freight_997_87464-berlin_1,freight,walk-freight-walk,87167,54169,walk-freight-walk,87167,54169,brandenburg,"berlin_outside_umweltzone,berlin_umweltzone,br...",berlin_umweltzone
freight_998_87464-berlin_1,freight,walk-freight-walk,87167,90464,walk-freight-walk,87167,90464,brandenburg,"berlin_outside_umweltzone,brandenburg",berlin_outside_umweltzone
freight_999_87464-berlin_1,freight,walk-freight-walk,87167,88992,walk-freight-walk,87167,88992,brandenburg,"berlin_outside_umweltzone,brandenburg",berlin_outside_umweltzone


In [41]:
# check differences of start_link between trips.csv and berlinwise.csv, 0 results is good
joined[joined["start_link_output"].notnull() &
       (joined["start_link_output"].astype(joined["start_link_analysis"].dtype) != joined["start_link_analysis"])]

,main_mode,modes_output,start_link_output,end_link_output,modes_analysis,start_link_analysis,end_link_analysis,start_area,visited_areas,end_area
trip_id,,,,,,,,,,


In [43]:
# check differences of end_link between trips.csv and berlinwise.csv, 0 results is good
joined[joined["end_link_output"].notnull() &
       (joined["end_link_output"].astype(joined["end_link_analysis"].dtype) != joined["end_link_analysis"])]

,main_mode,modes_output,start_link_output,end_link_output,modes_analysis,start_link_analysis,end_link_analysis,start_area,visited_areas,end_area
trip_id,,,,,,,,,,


In [44]:
# check differences of mode between trips.csv and berlinwise.csv
joined[joined["modes_output"].astype(joined["modes_analysis"].dtype) != joined["modes_analysis"]]

,main_mode,modes_output,start_link_output,end_link_output,modes_analysis,start_link_analysis,end_link_analysis,start_area,visited_areas,end_area
trip_id,,,,,,,,,,
100024301_1,pt,walk-pt-walk,117737,71096,walk-car-walk,117737,71096,brandenburg,brandenburg,brandenburg
100024301_2,pt,walk-pt-walk,71096,117737,walk-car-walk,71096,117737,brandenburg,brandenburg,brandenburg
100027001_1,pt,walk-pt-walk-pt-walk,71096,114042,walk-car-walk,71096,114042,brandenburg,"berlin_outside_umweltzone,brandenburg",berlin_outside_umweltzone
100027001_2,pt,walk-pt-walk-pt-walk,114042,71096,walk-car-walk,114042,71096,berlin_outside_umweltzone,"berlin_outside_umweltzone,brandenburg",brandenburg
100027101_1,bicycle,bicycle,46691,46693,walk,46691,46693,brandenburg,brandenburg,brandenburg
...,...,...,...,...,...,...,...,...,...,...
99990201_3,walk,walk,69410,120662,bicycle,69410,120662,brandenburg,brandenburg,brandenburg
99990201_4,walk,walk,120662,69410,bicycle,120662,69410,brandenburg,brandenburg,brandenburg
99998601_1,pt,walk-pt-walk,114789,121587,bicycle,114789,121587,brandenburg,brandenburg,brandenburg


Generate `ALL.trips_berlinwise.csv`

In [25]:
trips_berlinwise = generate_trips_berlinwise(agent_subpopulations.relevant, trips_berlinwise_by_scenario)
trips_berlinwise.to_csv(SCENARIOS_PATH / "ALL.trips_berlinwise.csv", sep=";")
trips_berlinwise

,person,trip_number,BASE.berlinwise,GR-HS.berlinwise,GR-WS.berlinwise,KR-HS.berlinwise,KB.berlinwise,MV.berlinwise,S1.berlinwise,S2.berlinwise,S3.berlinwise
trip_id,,,,,,,,,,,
10000001_1,10000001,1,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin
10000001_2,10000001,2,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin
10000001_3,10000001,3,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin
10000001_4,10000001,4,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin
10000001_5,10000001,5,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin
...,...,...,...,...,...,...,...,...,...,...,...
99996201_3,99996201,3,berlin_inner,berlin_inner,berlin_inner,berlin_inner,berlin_inner,berlin_inner,berlin_inner,berlin_inner,berlin_inner
99996201_4,99996201,4,berlin_orig,berlin_orig,berlin_orig,berlin_orig,berlin_orig,berlin_orig,berlin_orig,berlin_orig,berlin_orig
99998601_1,99998601,1,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin,non_berlin


Listet trips, die unterschiedliche berlinwise-Werte haben (z.B. von berlin_orig auf berlin_inner wechseln)

In [26]:
def berlinwise_differs_across_scenarios(row):
    values = {row[berlinwise_col(scenario)] for scenario in BerlinScenario}
    values.discard("cancelled")
    return len(values) != 1


trips_berlinwise[trips_berlinwise.apply(berlinwise_differs_across_scenarios, axis=1)]

,person,trip_number,BASE.berlinwise,GR-HS.berlinwise,GR-WS.berlinwise,KR-HS.berlinwise,KB.berlinwise,MV.berlinwise,S1.berlinwise,S2.berlinwise,S3.berlinwise
trip_id,,,,,,,,,,,


Generate `ALL.nonberlin_trips.csv`

In [27]:
nonberlin_trips = generate_nonberlin_trips(agent_subpopulations.relevant, trips_berlinwise_by_scenario)
nonberlin_trips.to_csv(SCENARIOS_PATH / "ALL.nonberlin_trips.csv", sep=";")
nonberlin_trips

,person,trip_number,BASE.berlin_transit,GR-HS.berlin_transit,GR-WS.berlin_transit,KR-HS.berlin_transit,KB.berlin_transit,MV.berlin_transit,S1.berlin_transit,S2.berlin_transit,S3.berlin_transit
trip_id,,,,,,,,,,,
10000001_1,10000001,1,other,other,other,other,other,other,other,other,other
10000001_2,10000001,2,other,other,other,other,other,other,other,other,other
10000001_3,10000001,3,other,other,other,other,other,other,other,other,other
10000001_4,10000001,4,other,other,other,other,other,other,other,other,other
10000001_5,10000001,5,other,other,other,other,other,other,other,other,other
...,...,...,...,...,...,...,...,...,...,...,...
99995901_4,99995901,4,other,other,other,other,other,other,other,other,other
99996201_1,99996201,1,other,other,other,other,other,other,other,other,other
99998601_1,99998601,1,other,other,other,other,other,other,other,other,other


Listet trips, die berlin_transit sind und einen teleportierten Modus (walk oder bicycle) haben.

In [17]:
joined = trips_by_scenario[BerlinScenario.GR_HS].join(trips_berlinwise_by_scenario[BerlinScenario.GR_HS],
                                                      "trip_id", "outer", lsuffix="output", rsuffix="analysis")
trips_with_teleported_transit = joined[
    (joined["berlinwise"] == "berlin_transit") & (joined["main_mode"].isin(["bicycle", "walk"]))]

In [18]:
trips_by_scenario[BerlinScenario.BASE].join(trips_with_teleported_transit[["start_linkoutput", "start_linkanalysis"]],
                                            how="inner")

,person,trip_number,dep_time,trav_time,wait_time,traveled_distance,euclidean_distance,main_mode,longest_distance_mode,modes,...,start_x,start_y,end_facility_id,end_link,end_x,end_y,first_pt_boarding_stop,last_pt_egress_stop,start_linkoutput,start_linkanalysis
108173201_3,108173201,3,18:18:37,01:17:17,00:00:00,14557,11197,bicycle,bicycle,bicycle,...,4.579772e+06,5.841566e+06,NaN,50131,4.588820e+06,5.834969e+06,NaN,NaN,14578,14578
108173201_4,108173201,4,20:03:01,01:17:17,00:00:00,14557,11197,bicycle,bicycle,bicycle,...,4.588820e+06,5.834969e+06,NaN,14578,4.579772e+06,5.841566e+06,NaN,NaN,50131,50131
118893501_1,118893501,1,10:24:13,01:51:50,00:00:00,21064,16515,bicycle,bicycle,bicycle,...,4.621037e+06,5.825435e+06,NaN,56594,4.605597e+06,5.831295e+06,NaN,NaN,105754,105754
124860301_3,124860301,3,17:06:36,00:58:49,00:00:00,11078,8521,bicycle,bicycle,bicycle,...,4.574878e+06,5.807235e+06,NaN,136217,4.583332e+06,5.806167e+06,NaN,NaN,2582,2582
126558801_1,126558801,1,12:27:45,00:28:12,00:00:00,5312,3841,bicycle,bicycle,bicycle,...,4.619114e+06,5.815260e+06,NaN,125423,4.619641e+06,5.811456e+06,NaN,NaN,98932,98932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95617401_2,95617401,2,10:46:16,00:25:16,00:00:00,4760,3661,bicycle,bicycle,bicycle,...,4.611716e+06,5.820404e+06,NaN,142972,4.612452e+06,5.823991e+06,NaN,NaN,141606,141606
95737701_5,95737701,5,11:43:26,00:38:44,00:00:00,7296,5612,bicycle,bicycle,bicycle,...,4.611033e+06,5.823609e+06,NaN,20968,4.612698e+06,5.818250e+06,NaN,NaN,141646,141646
95737701_6,95737701,6,12:22:10,00:38:44,00:00:00,7296,5612,bicycle,bicycle,bicycle,...,4.612698e+06,5.818250e+06,NaN,141646,4.611033e+06,5.823609e+06,NaN,NaN,20968,20968
95835801_6,95835801,6,21:08:00,00:10:05,00:00:00,1900,1461,bicycle,bicycle,bicycle,...,4.612507e+06,5.821896e+06,NaN,158621,4.611887e+06,5.823219e+06,NaN,NaN,110223,110223


Generates `agents_berlin_trips.csv`

In [30]:
agents_berlin_trips = generate_agents_berlin_trips(agent_subpopulations.relevant, trips_berlinwise)
agents_berlin_trips.to_csv(SCENARIOS_PATH / "agents_berlin_trips.csv", sep=";")
agents_berlin_trips

,executed_score,first_act_x,first_act_y,first_act_type,income,subpopulation,home-activity-zone,main_modes,berlin_trip
person,,,,,,,,,
10000001,90.723000,4.603071e+06,5.705496e+06,home_77400.0,2659.0,person,brandenburg,{car},no
1000001,116.991525,4.655520e+06,5.752325e+06,home_91800.0,826.0,person,brandenburg,{bicycle},no
100002101,92.331500,4.628160e+06,5.828612e+06,home_36000.0,826.0,person,brandenburg,{bicycle},yes
100002901,82.349915,4.628056e+06,5.830651e+06,home_48600.0,1847.0,person,brandenburg,{car},no
100004901,110.151148,4.625238e+06,5.824173e+06,home_54600.0,2070.0,person,brandenburg,{car},no
...,...,...,...,...,...,...,...,...,...
99994101,95.497982,4.632244e+06,5.826463e+06,home_60600.0,4329.0,person,brandenburg,{bicycle},no
99995401,117.394194,4.631520e+06,5.826864e+06,home_83400.0,1142.0,person,brandenburg,{bicycle},no
99995901,132.648340,4.627167e+06,5.827323e+06,home_77400.0,2332.0,person,brandenburg,"{walk, bicycle}",no


In [33]:
import pandas as pd

agents_berlin_trips_bak = pd.read_csv(SCENARIOS_PATH / "agents_berlin_trips.csv.bak", sep=";",
                                      dtype={"first_act_type": "category",
                                             "subpopulation": "category",
                                             "home-activity-zone": "category",
                                             })
agents_berlin_trips_bak

,executed_score,first_act_x,first_act_y,first_act_type,income,subpopulation,home-activity-zone,main_modes,berlin_trip
0,90.723000,4.603071e+06,5.705496e+06,home_77400.0,2659.0,person,brandenburg,{'car'},no
1,116.991525,4.655520e+06,5.752325e+06,home_91800.0,826.0,person,brandenburg,{'bicycle'},no
2,92.331500,4.628160e+06,5.828612e+06,home_36000.0,826.0,person,brandenburg,{'bicycle'},yes
3,82.349915,4.628056e+06,5.830651e+06,home_48600.0,1847.0,person,brandenburg,{'car'},no
4,110.151148,4.625238e+06,5.824173e+06,home_54600.0,2070.0,person,brandenburg,{'car'},no
...,...,...,...,...,...,...,...,...,...
408676,95.497982,4.632244e+06,5.826463e+06,home_60600.0,4329.0,person,brandenburg,{'bicycle'},no
408677,117.394194,4.631520e+06,5.826864e+06,home_83400.0,1142.0,person,brandenburg,{'bicycle'},no
408678,132.648340,4.627167e+06,5.827323e+06,home_77400.0,2332.0,person,brandenburg,"{'bicycle', 'walk'}",no
408679,109.114665,4.628290e+06,5.829095e+06,home_62400.0,1630.0,person,brandenburg,{'car'},yes


In [37]:
agents_berlin_trips_bak.index = agents_berlin_trips.index

In [38]:
agents_berlin_trips[agents_berlin_trips["berlin_trip"] != agents_berlin_trips_bak["berlin_trip"]]

,executed_score,first_act_x,first_act_y,first_act_type,income,subpopulation,home-activity-zone,main_modes,berlin_trip
person,,,,,,,,,
149744101,-247.247647,4.573301e+06,5.826848e+06,home_66600.0,1847.0,person,brandenburg,{},yes
29806301,-248.806012,4.605774e+06,5.847958e+06,home_68400.0,1399.0,person,brandenburg,{},yes
4400601,-276.631583,4.601988e+06,5.806979e+06,home_45600.0,2070.0,person,brandenburg,{},yes
50385901,-328.422616,4.610928e+06,5.781649e+06,home_40800.0,2070.0,person,brandenburg,{walk},yes
85932301,-272.815022,4.573582e+06,5.809714e+06,home_48000.0,2659.0,person,brandenburg,{},yes
87771201,-267.620696,4.572338e+06,5.808376e+06,home_51600.0,2070.0,person,brandenburg,{},yes
92958001,-261.604150,4.568177e+06,5.810418e+06,home_58200.0,3156.0,person,brandenburg,{walk},yes
93080801,-246.075311,4.576525e+06,5.805282e+06,home_71400.0,1399.0,person,brandenburg,{walk},yes
93203501,-261.017850,4.569940e+06,5.811894e+06,home_57000.0,1630.0,person,brandenburg,{},yes


Listet agents, die in der `home-activity-zone` berlin sind, aber gar keine trips in berlin haben. Hauptsächlich immobile Personen, aber auch 9 Grenzfälle

In [28]:
# sanity check
berlin_persons_no_berlin_trips = agents_berlin_trips[
    (agents_berlin_trips["home-activity-zone"] == "berlin") & (agents_berlin_trips["berlin_trip"] == "no") & (
            agents_berlin_trips["main_modes"] != set())]
berlin_persons_no_berlin_trips.to_csv(SCENARIOS_PATH / "berlin_persons_no_berlin_trips.csv", sep=";")
berlin_persons_no_berlin_trips

,executed_score,first_act_x,first_act_y,first_act_type,income,subpopulation,home-activity-zone,main_modes,berlin_trip
person,,,,,,,,,
301894401,142.648362,4.590714e+06,5.809078e+06,home_94200.0,4329.0,person,berlin,{walk},no
318691801,132.433802,4.583942e+06,5.810458e+06,home_73800.0,1630.0,person,berlin,{bicycle},no
347439501,140.784619,4.578045e+06,5.826504e+06,home_94800.0,1142.0,person,berlin,{walk},no
355350901,100.210245,4.610333e+06,5.823469e+06,home_47400.0,1847.0,person,berlin,{car},no
374525501,141.881706,4.577891e+06,5.825958e+06,home_94800.0,1630.0,person,berlin,{walk},no
394258901,136.727885,4.609866e+06,5.805978e+06,home_95400.0,1630.0,person,berlin,{bicycle},no
404370501,141.278379,4.577898e+06,5.825567e+06,home_96000.0,1399.0,person,berlin,{walk},no
414453401,136.676207,4.610516e+06,5.806352e+06,home_91200.0,2332.0,person,berlin,{bicycle},no
440394401,133.156617,4.612054e+06,5.805922e+06,home_94200.0,1142.0,person,berlin,{bicycle},no


In [33]:
immobile_persons = berlin_persons_no_berlin_trips = agents_berlin_trips[
    (agents_berlin_trips["main_modes"] == set()) & (agents_berlin_trips["home-activity-zone"] == "brandenburg")]
immobile_persons

,executed_score,first_act_x,first_act_y,first_act_type,income,subpopulation,home-activity-zone,main_modes,berlin_trip
person,,,,,,,,,
100009201,0.0,4.624434e+06,5.823701e+06,home_86400.0,4329.0,person,brandenburg,{},no
100035201,0.0,4.627736e+06,5.829616e+06,home_86400.0,2659.0,person,brandenburg,{},no
100074001,0.0,4.628630e+06,5.831094e+06,home_86400.0,1399.0,person,brandenburg,{},no
100120401,0.0,4.627844e+06,5.827000e+06,home_86400.0,1399.0,person,brandenburg,{},no
100135501,0.0,4.627831e+06,5.829134e+06,home_86400.0,1630.0,person,brandenburg,{},no
...,...,...,...,...,...,...,...,...,...
99819301,0.0,4.627360e+06,5.827827e+06,home_86400.0,2659.0,person,brandenburg,{},no
99822901,0.0,4.627475e+06,5.829040e+06,home_86400.0,2332.0,person,brandenburg,{},no
99870001,0.0,4.628073e+06,5.828305e+06,home_86400.0,3156.0,person,brandenburg,{},no
